# File for testing the initial generation pipeline for political ads

In [1]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures

# Specify Inputs

In [36]:
ad_length_time = 30
ad_length_words = 75

candidate = "Les Lascari"

party = "Republican"
position = "Congress"

constituency = "South Georgia"

"""
past achievements of candidate

statistics/sources

bills/laws/legislation the candidate supports
"""

talking_points = "Cracking down on illegal immigration, building Trump's wall"

target_audiences = ["Farmers and agricultural workers",
                    "Small business owners and entrepreneurs",
                    "Conservative-leaning voters"]

target_audience = target_audiences[0]

target_audience = "Republicans ages 40-60"


# Setup and Generate the first draft of the ad

In [37]:
pol_consultant_role = GPT.open_file("Prompts/political_consultant_role_v1.txt")

tags = {
    '<<CANDIDATE>>': candidate,
    '<<POSITION>>': position,
    '<<TALKING_POINTS>>': talking_points,
    '<<CONSTITUENCY>>': constituency,
    '<<TARGET_AUDIENCE>>': target_audience,
}

original_ad = GPT.generalized_gpt_prompt("Prompts/ad_generation_v1.txt", tags, role = pol_consultant_role)
print(original_ad)

Scene 1:
    - Dialogue: South Georgia, meet your next Congressman - Les Lascari! He'll FIGHT for what matters to YOU.
    - Cut Scene: Les Lascari standing in front of a rural South Georgia landscape, smiling and waving to the camera.

Scene 2:
    - Dialogue: Les Lascari knows we need to CRACK DOWN on illegal immigration - and he'll make it happen.
    - Cut Scene: Les Lascari talking with a concerned South Georgian family in their front yard.

Scene 3:
    - Dialogue: Les Lascari supports BUILDING Trump's wall - to protect our JOBS - and our FAMILIES.
    - Cut Scene: Les Lascari shaking hands with workers at a construction site.

Scene 4:
    - Dialogue: Vote Les Lascari for Congress - he's got your BACK, South Georgia.
    - Cut Scene: Les Lascari and his family waving at the camera, surrounded by cheering supporters.


# Create the Target Personas

In [35]:
# Load the persona gen role
persona_gen_role = GPT.open_file("Prompts/persona_generator_v1.txt")

# Create the personas
persona_text = GPT.generalized_gpt_prompt("Prompts/persona_input.txt", tags, role = persona_gen_role)

KeyboardInterrupt: 

In [ ]:
personas = persona_text.split('\n\n')
for persona in personas:
    print(persona)
    print('\n')

# Have each persona rate the ad

In [ ]:
def rate_persona(persona, ad):
    reviewer_role = GPT.open_file("Prompts/persona_rater_v1.txt").replace('<<PERSONA>>', persona)
    ad_review_in = GPT.open_file("Prompts/rate_ad_input.txt").replace('<<AD>>', ad)

    rating = GPT.chat_gpt(ad_review_in, temp=0.7, role=reviewer_role)
    return rating

In [ ]:
num_rounds = 2

ad = original_ad

# Preform a specified number of improvement rounds
for i in range(num_rounds):
    print("Round: ", str(i+1))
    print("Current ad: \n", ad)
    # Rate the ad with the personas
    ratings = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(rate_persona, persona, ad) for persona in personas]
        for future in concurrent.futures.as_completed(futures):
            rating = future.result()
            print(rating)
            print('\n')
            ratings.append(rating)     

    # Merge the reviews together
    merged_string = ""
    for i, review in enumerate(ratings, start=1):
        if i > 1:
            merged_string += " "
        merged_string += f"\n\nReview {i}: \n{review}"

    # Load the ad editor
    ad_editor_role = GPT.open_file("Prompts/ad_editor_v1.txt")

    # Load the editor in prompt
    edit_ad_in = GPT.open_file("Prompts/ad_edit_in.txt")
    edit_ad_in = edit_ad_in.replace('<<AD>>', ad)
    edit_ad_in = edit_ad_in.replace('<<FEEDBACK>>', merged_string)
    
    # Update the comercial to include the feedback
    updated_ad = GPT.chat_gpt(edit_ad_in, temp=0.7, role=ad_editor_role)
    
    ad = updated_ad

In [ ]:
print("Final Ad:", ad)

In [24]:
class Scene:
    def __init__(self, scene_number, dialogue="", cut_scene=""):
        self._scene_number = scene_number
        self._dialogue = dialogue
        self._cut_scene = cut_scene

    def get_scene_number(self):
        return self._scene_number

    def set_scene_number(self, scene_number):
        self._scene_number = scene_number

    def get_dialogue(self):
        return self._dialogue

    def set_dialogue(self, dialogue):
        self._dialogue = dialogue

    def get_cut_scene(self):
        return self._cut_scene

    def set_cut_scene(self, cut_scene):
        self._cut_scene = cut_scene

    def __str__(self):
        return f"Scene {self._scene_number}:\nDialogue: {self._dialogue}\nCut Scene: {self._cut_scene}"


In [25]:
def make_scenes(scene_text):
    scene_texts = scene_text.split('\n\n')
    scenes = []
    for i, text in enumerate(scene_texts):
        dialogue_start_index = text.find("Dialogue: ")
        cut_scene_start_index = text.find("Cut Scene: ")
        if dialogue_start_index == -1 or cut_scene_start_index == -1:
            continue
        dialogue = text[dialogue_start_index + len("Dialogue: "):cut_scene_start_index].strip()
        cut_scene = text[cut_scene_start_index + len("Cut Scene: "):].strip()
        scene = Scene(i+1, dialogue, cut_scene)
        scenes.append(scene)
    return scenes


In [26]:
scenes = make_scenes(updated_ad)

In [27]:
for s in scenes:
    print(s)
    print()

Scene 1:
Dialogue: "South Georgia, it's time to STAND TOGETHER - and PROTECT our homes, farms, and jobs."
    -
Cut Scene: Aerial shot of a beautiful South Georgia farm with American flags waving in the wind, vibrant color scheme.

Scene 2:
Dialogue: "Mr. Man will FIGHT for YOU - BALANCING immigration and our FARMING needs."
    -
Cut Scene: Montage of Mr. Man shaking hands with local farmers, agricultural workers, and talking with concerned constituents, interspersed with personal testimonials.

Scene 3:
Dialogue: "Together, we'll create REAL SOLUTIONS - addressing labor SHORTAGES while keeping our borders SECURE."
    -
Cut Scene: Mr. Man in a town hall meeting, listening intently to farmers and workers sharing their concerns, with text overlay of statistics and reliable sources.

Scene 4:
Dialogue: "Mr. Man - the EXPERIENCED, GENUINE voice South Georgia DESERVES."
    -
Cut Scene: Close-up of Mr. Man, confidently smiling and giving a thumbs up to the camera, with text overlay listin

In [28]:
from Utils.text_to_voice import create_voiceover

In [29]:
for s in scenes:
    create_voiceover(s.get_dialogue(), "scene" + str(s.get_scene_number()) +".mp3")

Text-to-speech conversion successful
Text-to-speech conversion successful
Text-to-speech conversion successful
Text-to-speech conversion successful
